In [3]:
import numpy as np
import pandas as pd
import requests
import json
import dotenv
import os

## Getting Biographical Data from the Congress API

The most important skill is reading the API documentation efficiently.

Look ONLY for these five things:

1. Root: the begining part of all URLs associated with this API for getting data back (I often find this in example first)

https://api.congress.gov/v3

2. Endpoint: the second part of the URL, generally it points to a specific data set to be returned

/member/{bioguideID}

The curly braces mean:fill this in for wichever member of congress you want

3. The headers: data that WE give to the ApI to identity ourselves, and provide othe information if requested
 * User Agent string: a string that identifies your software. If you are writing a Bot, what is you bot's name, and what methids are you using to access this API.

 The standard convertion is:

 'botname/version (email) python-requests/request-package-version'

In [5]:
botname = 'targ'
version = '0.0'
email = 'wna6mj@virginia.edu'
useragent = f'{botname}/{version} ({email}) python-requests/{requests.__version__}'
headers = {'User-Agent':useragent}
headers

{'User-Agent': 'targ/0.0 (wna6mj@virginia.edu) python-requests/2.32.5'}

4. API Parameters: there are ways to contro; what data is requested. The parameters are generally listed in the documentation for each endpoint. 3 kinds of parameters:

* Query parameters: should be listed in a dictionary and passed to the `params` argument of ` requests.get()`

* Path parameter: are part of the endpoint (such as bioguideId)

* Global parameters: apply to any use fo the API regardless if endpoint. Often that includes your API key. Can be in eith the headers ot the params

5. If an API key is needed, how to get one, and how will you let the API know the key?

API keys should be saved in a .env file in the same folder as your code. Use `dotenv` package to load the key into your code without ever displaying/exposing your key.

In [27]:
dotenv.load_dotenv()
congresskey =  os.getenv('congresskey')

In [28]:
params = {'format': 'json',
          'api_key': congresskey}

Put is all togethere

In [29]:
bioguideId = 'M001239'
root = 'https://api.congress.gov/v3'
endpoint = f'/member/{bioguideId}'

r = requests.get(root + endpoint,
                 headers = headers,
                 params = params)

r

<Response [200]>

In [30]:
r.json()

{'member': {'addressInformation': {'city': 'Washington',
   'district': 'DC',
   'officeAddress': '1013 Longworth House Office Building',
   'phoneNumber': '(202) 225-4711',
   'zipCode': 20515},
  'bioguideId': 'M001239',
  'birthYear': '1968',
  'cosponsoredLegislation': {'count': 170,
   'url': 'https://api.congress.gov/v3/member/M001239/cosponsored-legislation'},
  'currentMember': True,
  'depiction': {'attribution': 'Image courtesy of the Member',
   'imageUrl': 'https://www.congress.gov/img/member/67744ba20b34857ecc909149_200.jpg'},
  'directOrderName': 'John J. McGuire III',
  'district': 5,
  'firstName': 'John',
  'honorificName': 'Mr.',
  'invertedOrderName': 'McGuire, John J.',
  'lastName': 'McGuire',
  'middleName': 'J.',
  'officialWebsiteUrl': 'https://mcguire.house.gov/',
  'partyHistory': [{'partyAbbreviation': 'R',
    'partyName': 'Republican',
    'startYear': 2025}],
  'previousNames': [{'directOrderName': 'John McGuire',
    'endDate': '2025-01-10T17:36:18Z',
   

### Sponsored legislation

In [31]:
endpoint = f'/member/{bioguideId}/sponsored-legislation'
endpoint

'/member/M001239/sponsored-legislation'

In [32]:
params = {'format': 'json',
          'offset': 0,
          'limit': 250,
          'api_key': congresskey}

In [33]:
r = requests.get(root + endpoint,
                 headers=headers,
                 params=params)
r

<Response [200]>

In [34]:
myjson = r.json()

In [14]:
#myjson['sponsoredLegislation']

sponsoredLegislation_df = pd.json_normalize(myjson, record_path=['sponsoredLegislation'])

In [15]:
for x in sponsoredLegislation_df['title']:
    print(x)

To amend the Internal Revenue Code of 1986 to modify the tax on remittance transfers.
To change the name of the Department of Defense to the "Department of War", and for other purposes.
Border Wall Status Act
Make the District of Columbia Safe and Beautiful Act
FAIR Act
VA Data Transparency and Trust Act
Manned Aircraft Clarification Act
Uranium for Energy Independence Act of 2025
Agricultural and Forestry Hauling Efficiency Act


In [16]:
r = requests.get(sponsoredLegislation_df['url'][3],
                 headers=headers,
                 params=params)
r

<Response [200]>

In [17]:
r.json()

{'bill': {'actions': {'count': 6,
   'url': 'https://api.congress.gov/v3/bill/119/hr/5103/actions?format=json'},
  'committees': {'count': 2,
   'url': 'https://api.congress.gov/v3/bill/119/hr/5103/committees?format=json'},
  'congress': 119,
  'constitutionalAuthorityStatementText': '<pre>\n[Congressional Record Volume 171, Number 144 (Wednesday, September 3, 2025)]\n[House]\nFrom the Congressional Record Online through the Government Publishing Office [<a href="https://www.gpo.gov">www.gpo.gov</a>]\nBy Mr. McGUIRE:\nH.R. 5103.\nCongress has the power to enact this legislation pursuant\nto the following:\nArticle I, Section 8\n[Page H3829]\n</pre>',
  'cosponsors': {'count': 3,
   'countIncludingWithdrawnCosponsors': 3,
   'url': 'https://api.congress.gov/v3/bill/119/hr/5103/cosponsors?format=json'},
  'introducedDate': '2025-09-03',
  'latestAction': {'actionDate': '2025-09-10',
   'text': 'Ordered to be Reported (Amended) by the Yeas and Nays: 25 - 19.'},
  'legislationUrl': 'https:

### FEC Data for financial campaign contributions

In [125]:
dotenv.load_dotenv()
feckey = os.getenv('feckey')

In [126]:
root = 'https://api.open.fec.gov'
endpoint = '/v1/candidates/search/'

params = {'api_key': feckey,
          'q': 'John McGuire',
          'state': 'VA',
          'district': '5',
          'office': 'H',
          'year': '2024'}
r = requests.get(root + endpoint,
                 params=params,
                 headers=headers)
r

<Response [200]>

In [127]:
myjson = r.json()
candidate_id = myjson['results'][0]['candidate_id']
candidate_id

'H0VA07133'

In [128]:
endpoint = f'/v1/candidate/{candidate_id}/committees/'
params = {'api_key': feckey,
          'cycle': 2024}

r = requests.get(root + endpoint,
                 params=params,
                 headers=headers)
r   

<Response [200]>

In [129]:
[x['name'] for x in r.json()["results"]]

['MCGUIRE FOR VIRGINIA', 'MCGUIRE RECOUNT COMMITTEE']

In [130]:
committee_ids = [x['committee_id'] for x in r.json()['results']]
committee_ids

['C00856831', 'C00900118']

In [142]:
endpoint = '/v1/schedules/schedule_a/'
params = {'api_key': feckey,
          'committee_id': committee_ids[0],
          'per_page': 100,
          'sort': '-contribution_receipt_amount'}

r = requests.get(root + endpoint,
                 params=params,
                 headers=headers)
r

<Response [200]>

In [143]:
contrib_dict = [{'contributor_name': x['contributor_name'],
                'contributor_aggregate_ytd': x['contributor_aggregate_ytd'],
                'memo_text': x['memo_text'],
                'pdf_url': x['pdf_url']} for x in r.json()['results']]

pd.DataFrame(contrib_dict)


,contributor_name,contributor_aggregate_ytd,memo_text,pdf_url
0,MCGUIRE VICTORY FUND,279791.09,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
1,MCGUIRE VICTORY FUND,119484.10,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
2,MCGUIRE VICTORY FUND,397795.08,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
3,MCGUIRE VICTORY FUND,66488.21,TRANSFER FROM JOINT FUNDRAISER,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
4,MCGUIRE VICTORY FUND,279791.09,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
...,...,...,...,...
95,AMERICAN SECURITY PAC,5000.00,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
96,MAJORITY COMMITTEE PAC--MC PAC,10000.00,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
97,MAJORITY COMMITTEE PAC--MC PAC,10000.00,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
98,THE EYE OF THE TIGER PAC,10000.00,None,https://docquery.fec.gov/cgi-bin/fecimg/?20250...


In [133]:
lastindex = r.json()['pagination']['last_indexes']

In [134]:
lastindex

{'last_contribution_receipt_amount': '1000.00',
 'last_index': '4071720251209144543'}

In [120]:
params['last_contribution_receipt_amount']=lastindex['last_contribution_receipt_amount']
params['last_index']=lastindex['last_index']

In [121]:
r = requests.get(root + endpoint,
                 params=params,
                 headers=headers)
r

<Response [200]>

In [144]:
contrib_dict = [{'contributor_name': x['contributor_name'],
                'contributor_aggregate_ytd': x['contributor_aggregate_ytd'],
                'memo_text': x['memo_text'],
                'pdf_url': x['pdf_url']} for x in r.json()['results']]

pd.DataFrame(contrib_dict)

,contributor_name,contributor_aggregate_ytd,memo_text,pdf_url
0,MCGUIRE VICTORY FUND,279791.09,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
1,MCGUIRE VICTORY FUND,119484.10,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
2,MCGUIRE VICTORY FUND,397795.08,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
3,MCGUIRE VICTORY FUND,66488.21,TRANSFER FROM JOINT FUNDRAISER,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
4,MCGUIRE VICTORY FUND,279791.09,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
...,...,...,...,...
95,AMERICAN SECURITY PAC,5000.00,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
96,MAJORITY COMMITTEE PAC--MC PAC,10000.00,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
97,MAJORITY COMMITTEE PAC--MC PAC,10000.00,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
98,THE EYE OF THE TIGER PAC,10000.00,None,https://docquery.fec.gov/cgi-bin/fecimg/?20250...


### Loop over all contributions to McGuire

In [145]:
contrib_df = pd.DataFrame()

endpoint = '/v1/schedules/schedule_a/'
params = {'api_key': feckey,
          'committee_id': committee_ids[0],
          'per_page': 100,
          'sort': '-contribution_receipt_amount'}

r = requests.get(root + endpoint,
                 params=params,
                 headers=headers)

contrib_list = [{'contributor_name': x['contributor_name'],
                'contributor_aggregate_ytd': x['contributor_aggregate_ytd'],
                'memo_text': x['memo_text'],
                'pdf_url': x['pdf_url']} for x in r.json()['results']]
lastindex = r.json()['pagination']['last_indexes']

newrecords = len(r.json()['results'])

while newrecords > 0:
    print(len(contrib_list))

    params['last_contribution_receipt_amount']=lastindex['last_contribution_receipt_amount']
    params['last_index']=lastindex['last_index']

    r = requests.get(root + endpoint, params=params,headers=headers)

    contrib_list = contrib_list + [{'contributor_name': x['contributor_name'],
                'contributor_aggregate_ytd': x['contributor_aggregate_ytd'],
                'memo_text': x['memo_text'],
                'pdf_url': x['pdf_url']} for x in r.json()['results']]
    
    lastindex = r.json()['pagination']['last_indexes']

    newrecords = len(r.json()['results'])


contrib_df =pd.DataFrame(contrib_list)
contrib_df

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1771


,contributor_name,contributor_aggregate_ytd,memo_text,pdf_url
0,MCGUIRE VICTORY FUND,279791.09,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
1,MCGUIRE VICTORY FUND,119484.10,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
2,MCGUIRE VICTORY FUND,397795.08,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
3,MCGUIRE VICTORY FUND,66488.21,TRANSFER FROM JOINT FUNDRAISER,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
4,MCGUIRE VICTORY FUND,279791.09,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
...,...,...,...,...
1766,"REHNERT, GEOFFREY",7000.00,REDESIGNATION FROM,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
1767,"GOOD, JOHN P MR",7000.00,REDESIGNATION FROM,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
1768,"PEROT, ROSS",7000.00,MEMO: SPLIT OF-ROSS PEROT,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
1769,"MAFRIGE, DAVID Z.",6500.00,MEMO: SPLIT OF-DAVID Z. MAFRIGE,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
